In [ ]:
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt

## KMeans

In [ ]:
def general_kmeans(X: np.ndarray, k, toler=0.001):
    # centroids is a k x d matrix, where its values are the centroids of the clusters
    # C_i is a n x 1 vector, where its values are the cluster index of each point in X
    n = X.shape[0]
    # initialize random centroids
    centroids = X[np.random.choice(n, k, replace=False)]
    
    # until the centroids do not change significantly (dist(\micron_i^{t+1}, \micron_i^t) < toler)
    while True:
        # for each point x_j in X, assign it to the closest centroid \micron_i
        C_i = np.argmin(np.linalg.norm(X[:, None] - centroids, axis=-1), axis=-1)
        # we use argmin to get the index of the closest centroid

        # for each cluster C_i, update the centroid \micron_i as the mean of all points assigned to it
        micron_new = np.array([X[C_i == i].mean(axis=0) for i in range(k)])

        # if the centroids do not change significantly, break
        if np.linalg.norm(micron_new - centroids) < toler:
            break
        centroids = micron_new

    return centroids, C_i 

## KMeans++

In [ ]:
# kmeans++ is a variant of kmeans that initializes the centroids in a more intelligent way
# the first centroid is chosen uniformly at random from the data points

def kmeans_plus_plus(X: np.ndarray, k, toler=0.001):
    n = X.shape[0]
    # initialize centroids with first point
    centroids = [X[np.random.choice(n)]]
    
    for _ in range(k-1):
        # for each point x_j in X, calculate the distance to the closest centroid
        dist = np.linalg.norm(X[:, None] - centroids, axis=-1).min(axis=-1)
        # calculate the probability of each point to be chosen as the next centroid
        prob = dist**2 / (dist**2).sum()
        # choose the next centroid
        centroids.append(X[np.random.choice(n, p=prob)])
    
    centroids = np.array(centroids)

    # until the centroids do not change significantly (dist(\micron_i^{t+1}, \micron_i^t) < toler)
    while True:
        # for each point x_j in X, assign it to the closest centroid \micron_i
        C_i = np.argmin(np.linalg.norm(X[:, None] - centroids, axis=-1), axis=-1)
        # we use argmin to get the index of the closest centroid

        # for each cluster C_i, update the centroid \micron_i as the mean of all points assigned to it
        micron_new = np.array([X[C_i == i].mean(axis=0) for i in range(k)])

        # if the centroids do not change significantly, break
        if np.linalg.norm(micron_new - centroids) < toler:
            break
        centroids = micron_new

    return centroids, C_i

## MeanShift

In [ ]:
# meanshift require a KDTREE to find the nearest neighbors
from sklearn.neighbors import KDTree

def meanshift(X: np.ndarray, bandwidth=0.5, toler=0.001):
    n = X.shape[0]
    # assign each data point y_i = x_i for all points in X
    y = X.copy()
    # build a KDTree to find the nearest neighbors
    tree = KDTree(X)

    while True:
        # for each point y_i in X
        for i in range(n):
            # find the neighbors of y_i within a radius of bandwidth
            neighbors = tree.query_radius(y[i][None], bandwidth)[0]
            
            # update y_i  using the Gaussian kernel, computing the new value y_i as the weighted mean of the neighbors
            kernel_j = np.exp(-np.linalg.norm(y[neighbors] - y[i], axis=-1)**2 / (2 * bandwidth**2))
            y[i] = (kernel_j[:, None] * y[neighbors]).sum(axis=0) / kernel_j.sum()

            # if the centroids do not change significantly, break
        if np.linalg.norm(y - X) < toler:
            break
        X = y

    #group points y_i that converge to the same mode into the same cluster
    C_i = np.zeros(n, dtype=int)
    for i in range(n):
        C_i[i] = np.argmin(np.linalg.norm(y[i] - y, axis=-1))
    # assign each point x_i to the corresponding cluster based on its converged value y_i
    X = y
    return C_i

## Agglomerative Clustering (insecure)

In [ ]:
# agglomerative construye un dendograma con los datos, en donde cada punto es un cluster y a medida que se van uniendo los clusters, se va formando un árbol.
# def agglomerative(X: np.ndarray, metric='normal'):

## Divisive Clustering

In [ ]:
# divisive clustering divide el dendograma en clusters más pequeños a partir de un punto de corte dado
def DbisiveClustering(X: np.ndarray, var_tol=0.001):
    n = X.shape[0]
    # begin all data points in a single cluster: C = {X}
    C = [X]
    # while tha variance of the clusters is greater than a threshold
    variance = [np.var(C_i, axis=0) for C_i in C]
    while np.var(variance) > var_tol:
        # choose the cluster C_max with the largest dissimilarity (example: highest variance)
        C_max = np.argmax(variance)
        # split C_max into two clusters C_i and C_j with a partitioning algorithm (kmeans)
        _, C_i = general_kmeans(C[C_max], 2)
        # update clusters C = C - {C_max} + {C_i, C_j}
        C.pop(C_max)
        C.append(C[C_max][C_i == 0])
        C.append(C[C_max][C_i == 1])
        # update variance
        variance = [np.var(C_i, axis=0) for C_i in C]

    # assign each point x_i to the corresponding cluster based on the cluster it belongs to
    C_i = np.zeros(n, dtype=int)
    for i, C_i_ in enumerate(C):
        C_i[C_i_] = i
    return C_i


## Spectral Clustering

In [ ]:
# Spectral clustering aplica kmeans a una representación de los datos en un espacio de menor dimensión a partir de la matriz de afinidad
def SpectralClustering(afinity: np.ndarray, k):
    # construct degree matrix
    D = np.diag(np.sum(afinity, axis=0))

    # compute the Laplacian matrix
    L = D - afinity

    # computer the eigenvectors of the Laplacian matrix
    eigvals, eigvecs = np.linalg.eigh(L)
    # get k eigenvectors from the k smallest eigenvalues (sort them first)
    X = eigvecs[:, np.argsort(eigvals)[:k]]

    # normalize each row of X to have unit length
    X /= np.linalg.norm(X, axis=-1)[:, None]

    # apply kmeans to the rows of X into k clusters 
    _, C_i = general_kmeans(X, k, toler=0.001)

    # assign the i-th node to the cluster corresponding to the i-th row of X
    name_rows = np.argsort(eigvals)[:k] # sort indices of the k smallest eigenvalues
    C_i = C_i[np.argsort(name_rows)] # sort the cluster indices to match the original order of the nodes
    
    return C_i

## Markov Clustering

In [ ]:
# Cadena de Markov es un modelo de secuencia de eventos en que la probabilidad de que ocurra un evento depende del evento anterior
# Markov Clustering: Utiliza una matriz de afinidad para construir una matriz de transición y simular un proceso de difusión en la red
def MarkovClustering(afinity: np.ndarray, p=2, r=2):
    "p: power parameter, r: inflation parameter"
    # normalize the rows of the affinity matrix
    afinity /= afinity.sum(axis=1)[:, None]

    # repeat the following steps until convergence
    while True:
        # Expansion: raise the matrix to the power p
        afinity = afinity ** p

        # Inflation: raise each element to the power r and normalize the rows
        afinity = (afinity ** r) / (afinity ** r).sum(axis=1)[:, None]

        # if the matrix does not change significantly, break
        if np.linalg.norm(afinity - afinity @ afinity) < 0.001:
            break
        
    # group the rows of the matrix into clusters
    C_i = np.argmax(afinity, axis=1)
    return C_i